In [1]:
%reload_ext autoreload
%autoreload 2
# Import Libraries
from lib.PreprocessingTK import *
import pandas
import numpy as np

In [2]:
# Read Data with Features
'''
   Buying       v-high, high, med, low
   Maint        v-high, high, med, low
   Doors        2, 3, 4, 5-more
   Persons      2, 4, more
   Lug Boot     small, med, big
   Safety       low, med, high
   Evaluation   unacc, acc, good, vgood
'''
featureNames=["Buying", "Maint", "Doors", "Persons", "Lug Boot", "Safety", "Evaluation"]
data = pandas.read_csv("../../data/Cars/car.data",
                  names=featureNames)
# Show original data frame
print("Original Data Frame")
data

Original Data Frame


,Buying,Maint,Doors,Persons,Lug Boot,Safety,Evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [3]:
# Convert ordinal data to integer
ordinalValueDict = {
    "Buying": {
        "v-high": 1,
        "high": 2,
        "med": 3,
        "low": 4
    },
    "Maint": {
        "v-high": 1,
        "high": 2,
        "med": 3,
        "low": 4
    },
    "Doors": {
        "5-more": 5
    },
    "Persons": {
        "more" : 4
    },
    "Lug Boot": {
        "small": 1,
        "med": 2,
        "big": 3
    },
    "Safety": {
        "low": 1,
        "med": 2,
        "high": 3
    }
}
for featureName in ordinalValueDict.keys():
    convertToOrdinal(data, featureName, ordinalValueDict[featureName], inplace=True)
# Show updated data frame
print("\nData Frame after converting ordinal values to integer")
data

NameError: name 'convertToOrdinal' is not defined

In [ ]:
# Partition data into folds
# Stratify by Y class label
k = 5
proportions = (0.75, 0.25) # Train / Test proportions
classColName = "Evaluation"
print(f"\nPartition data into {k} folds with train, test, and (Optional) validation sets")
print(f"Stratifying by values in column: {classColName}")
folds = partition(data, k, classificationColumnId=classColName)
for i in range(0, len(folds)):
    print(f"Fold {i}, size={len(folds[i])}")



In [ ]:
# Comment: Could be improved to O(1) by assigning values directly, but this is more general
# e.g. classLabels = [y0, y1, y2, e.t.c.]
# Test our learner
classLabels = np.unique(data[classColName])
className = "Evaluation"
foldEvaluations = []
for i in range(0,k):
    testingSet = folds.pop(i)
    trainingSet = pandas.concat(folds, ignore_index=True)
    folds.insert(i, testingSet)
    foldEvaluation = {}
    for classLabel in classLabels:
        prediction = naivePredictor(trainingSet, classificationColId=className, method="classification")
        predicted_scores = [prediction for x in range(0,len(testingSet))] # Using first mode only

        accuracy = evaluateError(predicted_scores, testingSet[className], method="accuracy", classLabel=classLabel)
    # precision = evaluateError(predicted_scores, testingSet["class"], method="precision", classLabel=classLabel)
    # recall = evaluateError(predicted_scores, testingSet["class"], method="recall", classLabel=classLabel)
    # f1 = evaluateError(predicted_scores, testingSet["class"], method="f1", classLabel=classLabel)

        foldEvaluation[f'accuracy-{classLabel}'] = accuracy
    foldEvaluations.append(foldEvaluation)

print("\nLearning Performance Evaluation")
evalDf = pandas.DataFrame(foldEvaluations)
# evalDf.index.name = 'Fold'
evalDf = evalDf.rename_axis(index=None, columns='Fold')
evalDf.round(3)

In [ ]:
import numpy as np

for classLabel in classLabels:
    entry = "accuracy-" + classLabel
    avgAcc = np.mean(evalDf[entry])
    print(f"Average Mean Squared Error {classLabel}: {avgAcc}")